# Imports

In [ ]:
import pandas as pd
import numpy as np
import os # DEBUG
from glob import glob
from pandas_profiling import ProfileReport
import yaml
import re
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler

import category_encoders
%matplotlib inline

# Config

In [ ]:
PATH_ABS_SRC = os.getcwd()
PATH_REL = os.path.dirname(os.getcwd())

In [ ]:
with open("naming.yaml") as stream:
    naming = yaml.safe_load(stream)

# Load df

In [ ]:
df = pd.read_csv('../data/profiles_revised.csv')

In [ ]:
# CONSTS
ZODIAC_STRING_REPLACMENT = '&rsquo;' # corresponds to " ' "
cols = df.columns.tolist()
cols

# Cleaning

In [ ]:
# Using standard scaler
def std_scaler(df, col_names):
    scaled_features = df.copy()
 
    features = scaled_features[col_names]
    scaler = StandardScaler().fit(features.values)
    features = scaler.transform(features.values)
 
    scaled_features[col_names] = features

    return scaled_features


# Using min/max scaler
def minmax_scaler(df, col_names):
    scaled_features = df.copy()
 
    features = scaled_features[col_names]
    scaler = MinMaxScaler().fit(features.values)
    features = scaler.transform(features.values)
 
    scaled_features[col_names] = features

    return scaled_features

## Age

In [ ]:
# Remove nan's
df.dropna(inplace=True, subset=['age'])

# Scale
df = std_scaler(df, ['age'])


# Body Type

In [ ]:
# Remove nan's
df.dropna(inplace=True, subset=['body_type'])

# Encode body type
body_type_encoder = LabelEncoder()
body_type_encoder.fit(df['body_type'])
encoded_col_body_type = body_type_encoder.transform(df['body_type'])
df['body_type'] = encoded_col_body_type

# Todo: Consultation whether mapping (clearly unhealthy => -1, not optimal/unknown => 0, else => +1) justifiable 

# Diet

In [ ]:
# Remove nan's
df.dropna(inplace=True, subset=['diet'])

# Extract only diet
df['diet_extracted'] = df['diet'].str.split(' ').str[-1]

# Extract diet modifier
df['diet_modifier_extracted'] = df['diet'].str.split(' ').str[:-1]
df['diet_modifier_extracted'] = df['diet_modifier_extracted'].apply(lambda y: '' if len(y)==0 else y[0]) # replace empty lists with '' and extract term from list

# Todo: Consultation whether further mapping makes sense

# Encode diet
diet_encoder = LabelEncoder()
diet_encoder.fit(df['diet_extracted'])
encoded_col_diet = diet_encoder.transform(df['diet_extracted'])
df['diet'] = encoded_col_diet

# Encode diet modifier
diet_modifier_encoder = LabelEncoder()
diet_modifier_encoder.fit(df['diet_modifier_extracted'])
encoded_col_diet_modifier = diet_modifier_encoder.transform(df['diet_modifier_extracted'])
df['diet_modifier'] = encoded_col_diet_modifier


# Drop reduandant cols
df = df.drop('diet_extracted', axis=1)
df = df.drop('diet_modifier_extracted', axis=1)

# Drinks

In [ ]:
# Remove nan's
df.dropna(inplace=True, subset=['drinks'])

# Encode drinks modifier
drinks_encoder = LabelEncoder()
drinks_encoder.fit(df['drinks'])
encoded_col_drinks = drinks_encoder.transform(df['drinks'])
df['drinks'] = encoded_col_drinks

# Drugs

In [ ]:
# Remove nan's
df.dropna(inplace=True, subset=['drugs'])

# Encode drugs modifier
drinks_encoder = LabelEncoder()
drinks_encoder.fit(df['drugs'])
encoded_col_drugs = drinks_encoder.transform(df['drugs'])
df['drugs'] = encoded_col_drugs

# Education

In [ ]:
# Remove nan's
df.dropna(inplace=True, subset=['education'])


# Extract only education institution
# todo find better solution to use the dedicated mapper in naming.yaml
def education_institution_mapper(x):
    if 'college/university' in x:
        return 'college/university'
    if 'two-year college' in x:
        return 'two-year college'
    if 'masters program' in x:
        return 'masters program'
    if 'ph.d program' in x:
        return 'ph.d program'
    if 'high school' in x:
        return 'high school'
    if 'law school' in x:
        return 'law school'
    if 'med school' in x:
        return 'med school'
    if 'space camp' in x:
        return 'space camp'

# Extract only education status
def education_status_mapper(x):
    if 'dropped out of' in x:
        return 'dropped out of'
    if 'working on' in x:
        return 'working on'
    if 'graduated from' in x:
        return 'graduated from'


df['education_status_extracted'] = df['education'].apply(lambda x: education_status_mapper(x))
df['education_institution_extracted'] = df['education'].apply(lambda x: education_institution_mapper(x))


# Encode education_status
education_status_encoder = LabelEncoder()
education_status_encoder.fit(df['education_status_extracted'])
encoded_col_education_status = education_status_encoder.transform(df['education_status_extracted'])
df['education_status_extracted'] = encoded_col_education_status

# Encode diet modifier
education_institution_encoder = LabelEncoder()
education_institution_encoder.fit(df['education_institution_extracted'])
encoded_col_education_institution = education_institution_encoder.transform(df['education_institution_extracted'])
df['education_institution_extracted'] = encoded_col_education_institution

# Drop reduandant cols
df = df.drop('education', axis=1)
df

# Ethnicity

# Height

In [ ]:
# Remove nan's
df.dropna(inplace=True, subset=['height'])

# Scale
df = std_scaler(df, ['height'])

# Income

In [ ]:
# Replace -1 entries
df['income'] = df['income'].apply(lambda y: np.nan if y==-1 else y) # replace -1 with nan
# Todo: Maybe insert non nan but average income (only 5k values after that)

# Remove nan's
df.dropna(inplace=True, subset=['income'])

# Scale
df = std_scaler(df, ['income'])
df

# Job

In [ ]:
# Remove nan's
df.dropna(inplace=True, subset=['job'])

# Encode drugs modifier
job_encoder = LabelEncoder()
job_encoder.fit(df['job'])
encoded_col_job = job_encoder.transform(df['job'])
df['job'] = encoded_col_job
df

# Offspring

# Orientation

# Pets

# Religion

# Sex

In [ ]:
# Remove nan's
df.dropna(inplace=True, subset=['sex'])

# Encode drugs modifier
sex_encoder = LabelEncoder()
sex_encoder.fit(df['sex'])
encoded_col_sex = sex_encoder.transform(df['sex'])
df['sex'] = encoded_col_sex
df

# Sign

In [ ]:
# Remove nan's
df.dropna(inplace=True, subset=['sign'])


# Extract only sign
df['sign_extracted'] = df['sign'].str.split(' ').str[0]

# Extract sign modifier
df['sign_modifier_extracted'] = df['sign'].str.split(' ').str[1:]
df['sign_modifier_extracted'] = df['sign_modifier_extracted'].apply(lambda y: '' if len(y)==0 else y) # replace empty lists with ''
df['sign_modifier_extracted'] = df['sign_modifier_extracted'].apply(lambda y: ' '.join(y) if len(y)!=0 else y) # join list of strings together
df['sign_modifier_extracted'] = df['sign_modifier_extracted'].str.replace(ZODIAC_STRING_REPLACMENT,'\'')  # replace 

# Encode sign
sign_encoder = LabelEncoder()
sign_encoder.fit(df['sign_extracted'])
encoded_col_sign = sign_encoder.transform(df['sign_extracted'])
df['sign_extracted'] = encoded_col_sign

# Encode sign modifier
sign_modifier_encoder = LabelEncoder()
sign_modifier_encoder.fit(df['sign_modifier_extracted'])
encoded_col_sign_modifier = sign_modifier_encoder.transform(df['sign_modifier_extracted'])
df['sign_modifier_extracted'] = encoded_col_sign_modifier

# Drop reduandant cols
df = df.drop('sign', axis=1)
df

# Smokes

In [ ]:
# Remove nan's
df.dropna(inplace=True, subset=['smokes'])

# Encode smokes modifier
smokes_encoder = LabelEncoder()
smokes_encoder.fit(df['smokes'])
encoded_col_smokes = smokes_encoder.transform(df['smokes'])
df['smokes'] = encoded_col_smokes
df

# Speaks

# Status

In [ ]:
# Remove nan's
df.dropna(inplace=True, subset=['status'])

# Encode drugs modifier
status_encoder = LabelEncoder()
status_encoder.fit(df['status'])
encoded_col_status = status_encoder.transform(df['status'])
df['status'] = encoded_col_status
df